# 🥋 Lekcja 4: Advanced Indexing (Gather & Scatter)

Większość ludzi zna indeksowanie typu `x[0, 2]`.
Ale w Deep Learningu często musimy wybierać dane w sposób **dynamiczny**.

**Trzej Królowie Indeksowania:**
1.  **`index_select`:** Wybierz konkretne wiersze/kolumny (proste).
2.  **`gather`:** "Dla każdego wiersza wybierz element z innej kolumny". (Kluczowe w RL i NLP).
3.  **`scatter_`:** "Wstaw wartość w konkretne, nieregularne miejsce". (Kluczowe w One-Hot Encoding i Grafach).

Zrozumienie `gather` to moment "Aha!", który pozwala pisać wydajne funkcje straty bez pętli `for`.

In [1]:
import torch

# Dane: [Batch=3, Cechy=4]
x = torch.tensor([
    [10, 20, 30, 40], # Próbka 0
    [50, 60, 70, 80], # Próbka 1
    [90, 91, 92, 93]  # Próbka 2
])

print("--- DANE WEJŚCIOWE ---")
print(x)

--- DANE WEJŚCIOWE ---
tensor([[10, 20, 30, 40],
        [50, 60, 70, 80],
        [90, 91, 92, 93]])


## 1. `index_select` (Prosty wybór)

Chcemy wybrać np. kolumnę 0 i kolumnę 2 dla **wszystkich** wierszy.
To działa jak wyciąganie kart z talii.

Wymaga podania indeksów jako Tensora (Long/Int).

In [2]:
# Chcemy kolumnę 0 i 2
indices = torch.tensor([0, 2])

# dim=1 (kolumny)
selected = torch.index_select(x, dim=1, index=indices)

print("--- INDEX SELECT (Kolumny 0 i 2) ---")
print(selected)
# To samo co x[:, [0, 2]], ale często szybsze wewnątrz skomplikowanych funkcji

--- INDEX SELECT (Kolumny 0 i 2) ---
tensor([[10, 30],
        [50, 70],
        [90, 92]])


## 2. `gather` (Chirurgiczna precyzja)

To jest problem z **Reinforcement Learning (DQN)**.
Mamy Q-wartości dla wszystkich akcji: `[Lewo, Prawo, Skok, Strzał]`.
Agent w stanie 0 wybrał **Lewo** (index 0).
Agent w stanie 1 wybrał **Skok** (index 2).
Agent w stanie 2 wybrał **Strzał** (index 3).

Chcemy wyciągnąć wartość TYLKO dla wybranych akcji.
`index_select` tego nie zrobi (bo wybiera całe kolumny).

**Zasada `gather(dim, index)`:**
Dla każdego elementu w wymiarze `dim`, użyj wartości z `index` jako adresu.
Kształt tensora `index` determinuje kształt wyniku.

In [3]:
# Akcje wybrane przez agenta w 3 sytuacjach
# Musimy dodać wymiar, żeby pasował do x (Batch, 1)
actions = torch.tensor([
    [0], # W wierszu 0 weź kolumnę 0
    [2], # W wierszu 1 weź kolumnę 2
    [3]  # W wierszu 2 weź kolumnę 3
])

print(f"Indeksy akcji:\n{actions}")

# GATHER
# dim=1 oznacza: "Przesuwamy się po wierszach normalnie (0, 1, 2...), 
# ale numer kolumny bierzemy z tensora 'actions'"
picked_values = torch.gather(x, dim=1, index=actions)

print("\n--- GATHER (Wartości dla wybranych akcji) ---")
print(picked_values)

# Weryfikacja:
# Wiersz 0 -> index 0 -> wartość 10
# Wiersz 1 -> index 2 -> wartość 70
# Wiersz 2 -> index 3 -> wartość 93

Indeksy akcji:
tensor([[0],
        [2],
        [3]])

--- GATHER (Wartości dla wybranych akcji) ---
tensor([[10],
        [70],
        [93]])


## 3. `scatter_` (Odwrotność Gather)

Teraz w drugą stronę. Mamy puste płótno (zera) i listę indeksów.
Chcemy wstawić "jedynki" w te miejsca.
To klasyczny **One-Hot Encoding** robiony ręcznie.

Metoda kończy się na `_` (underscore), co w PyTorch oznacza **in-place** (modyfikuje tensor w pamięci, zamiast tworzyć nowy).

Wzór: `tensor.scatter_(dim, index, src)`

In [4]:
# Puste płótno [3 wiersze, 5 klas]
target = torch.zeros(3, 5)

# Prawdziwe klasy dla każdego wiersza
indices = torch.tensor([
    [2], # Wiersz 0 -> Klasa 2
    [0], # Wiersz 1 -> Klasa 0
    [4]  # Wiersz 2 -> Klasa 4
])

# Wartość do wstawienia (src)
value = 1.0

# Wstawiamy
target.scatter_(dim=1, index=indices, value=value)

print("--- SCATTER (One-Hot Encoding) ---")
print(target)

# Weryfikacja:
# Wiersz 0: [0, 0, 1, 0, 0] (Indeks 2 zapalony)

--- SCATTER (One-Hot Encoding) ---
tensor([[0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.]])


## 🥋 Black Belt Summary

Te funkcje są fundamentem pisania niestandardowych warstw i funkcji kosztu.

1.  **`index_select`**: Wycinanie "pasków" z tensora.
2.  **`gather`**: "Dla każdego wiersza, daj mi *jego* specyficzną kolumnę". (Kluczowe w DQN, Transformer Decoding).
3.  **`scatter_`**: "Rozrzuć wartości pod wskazane adresy". (Kluczowe w GNN - Message Passing, One-Hot).

**Tip:** `gather` i `scatter` wymagają, aby tensor indeksów miał ten sam wymiar (rank) co tensor danych. Dlatego użyliśmy `[[0], [2]]` (2D) a nie `[0, 2]` (1D).